### Generate ground truth results for T2B prompts 

In [1]:
import basico

c:\code\benchmark\AIAgents4Pharma\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Set 1

##### Precision Medicine Model

In [2]:
# generate expected answers for the basic prompts
# precision medicine model
mod_precision = basico.load_biomodel('BIOMD0000000537')
print(basico.get_model_name())
# run simulation for set 1 and get the ground truth answer
tc = basico.run_time_course(2016, 2016)
tc.iloc[-1]["CRP{serum}"]

Dwivedi2014 - Crohns IL6 Disease model - Anti-IL6R Antibody


2.2691352206218145

##### Epidemiology Model

In [3]:
# epidemiology mode
mod_epidemiology = basico.load_biomodel('BIOMD0000000971')
print(basico.get_model_name())

Tang2020 - Estimation of transmission risk of COVID-19 and impact of public health interventions


In [4]:
# run simulation for set 1 and get the ground truth answer
tc = basico.run_time_course(50, 100)
tc.iloc[-1]["Infected"]

104338.52373753088

##### Theoritical Model

In [5]:
# epidemiology mode
mod_theory = basico.load_biomodel('BIOMD0000000027')
print(basico.get_model_name())

Markevich2004 - MAPK double phosphorylation,  ordered Michaelis-Menton


In [6]:
tc = basico.run_time_course(1000, 1000)
tc.iloc[-1]["Mpp"]

48.176383215821375

### Set 2 vary simulation time, interval and species to be simulated

In [25]:
def run_precision_model(interval, time, species, concentration):
    basico.set_species(name = species, model = mod_precision, initial_concentration = concentration)
    # check if the species was set correctly
    #print(basico.get_species(name = species, model = mod_precision)['initial_concentration'])
    # run simulations
    tc = basico.run_time_course(interval, time, model = mod_precision)
    return tc.iloc[-1][["CRP{serum}",  "CRP{liver}", "IL6{serum}", "STAT3{gut}"]]

run_precision_model(2016, 2016, "CRP{serum}", 0.00001)


CRP{serum}    2.248250
CRP{liver}    0.788363
IL6{serum}    0.000643
STAT3{gut}    9.420788
Name: 2016.0, dtype: float64

In [21]:
basico.get_species(name = "CRP{liver}", model = mod_precision)['initial_concentration']

name
CRP    158.325847
Name: initial_concentration, dtype: float64

In [8]:
def run_epidemiology_model(interval, time):
    tc = basico.run_time_course(interval, time, model = mod_epidemiology)
    return tc.iloc[-1][["Infected", "Susceptible", "Recovered", "Hospitalised"]]


In [9]:
def run_theory_model(interval, time):
    tc = basico.run_time_course(interval, time, model = mod_theory)
    return tc.iloc[-1][["Mpp", "M", "Mp"]]


In [22]:
# print intial concentrations of species
for species in ["CRP{serum}",  "CRP{liver}", "IL6{serum}", "STAT3{gut}"]:
    print(basico.get_species(name = species, model = mod_precision)['initial_concentration'])


name
CRP    0.00001
Name: initial_concentration, dtype: float64
name
CRP    158.325847
Name: initial_concentration, dtype: float64
name
IL6    0.000436
Name: initial_concentration, dtype: float64
name
STAT3    0.610636
Name: initial_concentration, dtype: float64


In [41]:
initial_concentr_IL6_serum.iloc[0]

435.6288965511659

In [44]:
# Run with different random conditions and store in dictionary
results = {}
# adjust initial concentrations of species
initial_concentr_CRP_serum = basico.get_species(name = "CRP{serum}", model = mod_precision)['initial_concentration'].iloc[0]
initial_concentr_CRP_liver = basico.get_species(name = "CRP{liver}", model = mod_precision)['initial_concentration'].iloc[0]
initial_concentr_IL6_serum = basico.get_species(name = "IL6{serum}", model = mod_precision)['initial_concentration'].iloc[0]
initial_concentr_STAT3_gut = basico.get_species(name = "STAT3{gut}", model = mod_precision)['initial_concentration'].iloc[0]
# time 
conditions = [
    (2016, 2016, "CRP{serum}", 0.01),
    (20, 100, "CRP{serum}", 1000),
    (2016, 4032, "CRP{serum}", 2.6),
    (1000, 1000, "IL6{serum}", initial_concentr_IL6_serum*10), 
    (2016, 1000, "CRP{liver}", initial_concentr_CRP_liver*10),
    (500, 500, "STAT3{gut}", initial_concentr_STAT3_gut*10),
    (1500, 2000, "STAT3{gut}", initial_concentr_STAT3_gut*0.10)
]

for time, interval, species, concentration in conditions:
    key = f"interval_{interval}_time_{time}_species_{species}_concentration_{concentration}"
    print (concentration)
    results[key] = run_precision_model(interval, time, species, concentration)

results

0.01
1000
2.6
4356.288965511659
1583.2584678161063
6.106360135082121e-06
6.106360135082121e-08


{'interval_2016_time_2016_species_CRP{serum}_concentration_0.01': CRP{serum}    10.019164
 CRP{liver}     6.492506
 IL6{serum}     0.000641
 STAT3{gut}     9.133764
 Name: 2016.0, dtype: float64,
 'interval_100_time_20_species_CRP{serum}_concentration_1000': CRP{serum}    226.440005
 CRP{liver}    162.014501
 IL6{serum}      0.000676
 STAT3{gut}      0.332824
 Name: 100.0, dtype: float64,
 'interval_4032_time_2016_species_CRP{serum}_concentration_2.6': CRP{serum}    221.060568
 CRP{liver}    158.323640
 IL6{serum}      0.000436
 STAT3{gut}      0.596129
 Name: 4032.0, dtype: float64,
 'interval_1000_time_1000_species_IL6{serum}_concentration_4356.288965511659': CRP{serum}    91.229032
 CRP{liver}    60.845307
 IL6{serum}     0.000642
 STAT3{gut}     6.815264
 Name: 1000.0, dtype: float64,
 'interval_1000_time_2016_species_CRP{liver}_concentration_1583.2584678161063': CRP{serum}    91.229207
 CRP{liver}    60.845411
 IL6{serum}     0.000642
 STAT3{gut}     6.815259
 Name: 1000.0, dtype:

In [11]:
# Run with different random conditions and store in dictionary
results = {}
# time 
conditions = [
    (50, 100),
    (100, 200),
    (180, 400),
    (500, 1000), 
    (20, 400),
    (10, 10)
]

for time, interval in conditions:
    key = f"interval_{interval}_time_{time}"
    results[key] = run_epidemiology_model(interval, time)

results

{'interval_100_time_50': Infected        7.143353e+04
 Susceptible     1.055609e+06
 Recovered       4.586298e+06
 Hospitalised    8.688262e+04
 Name: 100.0, dtype: float64,
 'interval_200_time_100': Infected        3.494414e+04
 Susceptible     1.055585e+06
 Recovered       7.364926e+06
 Hospitalised    4.248752e+04
 Name: 200.0, dtype: float64,
 'interval_400_time_180': Infected        8.361387e+03
 Susceptible     1.055585e+06
 Recovered       9.388977e+06
 Hospitalised    1.016636e+04
 Name: 400.0, dtype: float64,
 'interval_1000_time_500': Infected        1.145493e+02
 Susceptible     1.055585e+06
 Recovered       1.001690e+07
 Hospitalised    1.392771e+02
 Name: 1000.0, dtype: float64,
 'interval_400_time_20': Infected        8.361387e+03
 Susceptible     1.055585e+06
 Recovered       9.388977e+06
 Hospitalised    1.016636e+04
 Name: 400.0, dtype: float64,
 'interval_10_time_10': Infected        3.413654e+03
 Susceptible     1.079620e+07
 Recovered       2.980390e+03
 Hospitalise

In [12]:
# Run with different random conditions and store in dictionary
results = {}
# time 
conditions = [
    (1000, 100),
    (1000, 1000),
    (180, 400),
    (500, 1000), 
    (20, 400),
    (10, 10)
]

for time, interval in conditions:
    key = f"interval_{interval}_time_{time}"
    results[key] = run_theory_model(interval, time)

results

{'interval_100_time_1000': Mpp     18.763213
 M      474.237334
 Mp       6.999453
 Name: 100.0, dtype: float64,
 'interval_1000_time_1000': Mpp     48.176383
 M      439.166551
 Mp      12.657066
 Name: 1000.0, dtype: float64,
 'interval_400_time_180': Mpp     39.992150
 M      448.718444
 Mp      11.289406
 Name: 400.0, dtype: float64,
 'interval_1000_time_500': Mpp     48.176389
 M      439.166544
 Mp      12.657067
 Name: 1000.0, dtype: float64,
 'interval_400_time_20': Mpp     39.992150
 M      448.718444
 Mp      11.289406
 Name: 400.0, dtype: float64,
 'interval_10_time_10': Mpp      1.614307
 M      496.083772
 Mp       2.301920
 Name: 10.0, dtype: float64}